In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
from sklearn.cluster import MeanShift
# Import CMasher to register colormaps
import cmasher as cmr
import geopandas as gpd
import cftime
import glob
from scipy.stats import linregress
from decimal import Decimal, ROUND_DOWN
import statsmodels.api as sm
import statsmodels.formula.api as smf

/home/estudiantes/anaconda/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
#Cargando los datos de ERA5-Land
# Usar glob para seleccionar todos los archivos CSV en la ruta
archivos_nc = sorted(glob.glob("/home/andreagvc/data/ERA/ERA/ERA_*_hourly_rec*.nc"))

#Combinando los datos de ERA
ds_combinado=xr.open_mfdataset(archivos_nc,combine="by_coords")

In [3]:
ds_combinado

<xarray.Dataset> Size: 577MB
Dimensions:    (sid: 90, time: 394464)
Coordinates:
  * time       (time) datetime64[ns] 3MB 1979-01-01 ... 2023-12-31T23:00:00
    latitude   (sid) float64 720B dask.array<chunksize=(90,), meta=np.ndarray>
    longitude  (sid) float64 720B dask.array<chunksize=(90,), meta=np.ndarray>
    number     int64 8B 0
    expver     (time) <U4 6MB dask.array<chunksize=(8760,), meta=np.ndarray>
  * sid        (sid) int64 720B 760013 760053 760400 ... 768480 769030 769043
Data variables:
    TWB_d      (sid, time) float64 284MB dask.array<chunksize=(90, 8760), meta=np.ndarray>
    T_d        (sid, time) float32 142MB dask.array<chunksize=(90, 8760), meta=np.ndarray>
    q_d        (sid, time) float32 142MB dask.array<chunksize=(90, 8760), meta=np.ndarray>

In [4]:
#Seleccionando las variables

Tw_ERA=ds_combinado.TWB_d
T_ERA=ds_combinado.T_d
q_ERA=ds_combinado.q_d

In [5]:
#Cargando los datos de las estaciones
df=pd.read_csv("IDD.csv",header=None)
ID=df.loc[(df[0]>="760011-99999")&(df[0]<="780160-13601")] 
ID1=ID[0].str.split(pat="-", expand=True) #Separando 
ID["ID"]=ID1[0].astype(int)

esta=pd.read_fwf("ID.txt",header=None)
p=esta[1]
filtered_df = ID[(ID["ID"]).isin(p)]
filtered_df #Encontrando los datos de las estaciones que ocupamos
lat=filtered_df[2].values
lon=filtered_df[3].values
ele=filtered_df[4].values

#Agregando la elevación al dataframe
esta["elevacion"]=ele
esta["Lat"]=lat
esta["Lon"]=lon
esta["Lugar"]=filtered_df[1].values

/tmp/ipykernel_177049/1436073433.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ID["ID"]=ID1[0].astype(int)


In [6]:
#Cargando los datos de las estaciones Had_ISD
da_list=[]
for i,ii in enumerate(esta[1]):
    datos=xr.open_dataset(f"/home/andreagvc/data/ERA/ERA/Datos_HAD/{ii}99999.nc")
    datos=datos.assign_coords(estacion=ii)
    datos=datos.expand_dims('estacion')
    da_list.append(datos)

In [7]:
HAD=xr.concat(da_list,dim='estacion')

In [8]:
#Seleccionando las variables promedio  de HadISD
HAD_Tw=(HAD.Tw.where(HAD.Tw.time.dt.year>1978, drop=True))
HAD_T=(HAD.T.where(HAD.T.time.dt.year>1978, drop=True))
HAD_SH=(HAD.SH.where(HAD.SH.time.dt.year>1978, drop=True))


In [9]:
time_vec=(HAD.Tw.where(HAD.Tw.time.dt.year>1978, drop=True)).resample(time="YE").mean()

In [10]:
def PER95(datos,time_vec,tipo=None):
    P95=np.zeros(time_vec.shape) #Haciendo un array para rellenar los datos, dado que la rutina dos es más rápida, se ocupará ese método para calcular TW
    
    for Id,estacion in enumerate(HAD_Tw.estacion):
        if tipo==0:
            slice_data=datos.isel(estacion=Id)
        else:
            slice_data=datos.isel(sid=Id)
        for it,time in enumerate(time_vec.time.dt.year):
            datarec=slice_data.loc[slice_data.time.dt.year==time]
            if datarec.isnull().all():
                P95[Id,it]=np.nan
                continue
            datarec=datarec.dropna(dim='time')
            p95=np.percentile(datarec,99)
            #print(time,p95)
            # checamos si esta coordenada tiene datos o no, si son nulos, no calculamos nada
            P95[Id,it]=p95
    
    #print(np.nanmean(P95,axis=0))
    return P95

In [11]:
HAD_Tw_p=PER95(HAD_Tw,time_vec,tipo=0)
HAD_T_p=PER95(HAD_T,time_vec,tipo=0)
HAD_SH_p=PER95(HAD_SH,time_vec,tipo=0)

In [12]:
HAD_Tw_P95 = xr.DataArray(HAD_Tw_p, dims=("estacion", "time"),   #Haciando un xarray.DataArray para poder encontrar de forma más rápida 
                                                                    #los datos de TWmax,TWmean,Tmax usando resample
                   coords={"estacion":HAD_Tw.estacion,"time": time_vec.time.dt.year},
                   attrs={"units": "K"})

HAD_T_P95 = xr.DataArray(HAD_T_p, dims=("estacion", "time"),   #Haciando un xarray.DataArray para poder encontrar de forma más rápida 
                                                                    #los datos de TWmax,TWmean,Tmax usando resample
                   coords={"estacion":HAD_Tw.estacion,"time": time_vec.time.dt.year},
                   attrs={"units": "K"})

HAD_SH_P95 = xr.DataArray(HAD_SH_p, dims=("estacion", "time"),   #Haciando un xarray.DataArray para poder encontrar de forma más rápida 
                                                                    #los datos de TWmax,TWmean,Tmax usando resample
                   coords={"estacion":HAD_Tw.estacion,"time": time_vec.time.dt.year},
                   attrs={"units": "K"})

In [ ]:
Tw_ERA_p=PER95(Tw_ERA,time_vec,tipo=1)
T_ERA_p=PER95(T_ERA,time_vec,tipo=1)
q_ERA_p=PER95(q_ERA,time_vec,tipo=1)

In [ ]:
ERA_Tw_P95 = xr.DataArray(Tw_ERA_p, dims=("estacion", "time"),   #Haciando un xarray.DataArray para poder encontrar de forma más rápida 
                                                                    #los datos de TWmax,TWmean,Tmax usando resample
                   coords={"estacion":HAD_Tw.estacion,"time": time_vec.time.dt.year},
                   attrs={"units": "K"})

ERA_T_P95 = xr.DataArray(T_ERA_p, dims=("estacion", "time"),   #Haciando un xarray.DataArray para poder encontrar de forma más rápida 
                                                                    #los datos de TWmax,TWmean,Tmax usando resample
                   coords={"estacion":HAD_Tw.estacion,"time": time_vec.time.dt.year},
                   attrs={"units": "K"})

ERA_SH_P95 = xr.DataArray(q_ERA_p, dims=("estacion", "time"),   #Haciando un xarray.DataArray para poder encontrar de forma más rápida 
                                                                    #los datos de TWmax,TWmean,Tmax usando resample
                   coords={"estacion":HAD_Tw.estacion,"time": time_vec.time.dt.year},
                   attrs={"units": "K"})

In [ ]:
#Promediando el P95 por estacion
HAD_Tw_P95_mean=HAD_Tw_P95.mean("estacion")
HAD_T_P95_mean=HAD_T_P95.mean("estacion")
HAD_SH_P95_mean=HAD_SH_P95.mean("estacion")

ERA_Tw_P95_mean=ERA_Tw_P95.mean("estacion")
ERA_T_P95_mean=ERA_T_P95.mean("estacion")
ERA_SH_P95_mean=ERA_SH_P95.mean("estacion")



In [ ]:
fig = plt.figure(figsize=(8,8),dpi=150)

plt.subplot(3,1,1)

x=(HAD_Tw_P95_mean).time
y=HAD_Tw_P95_mean-273.15

Tiempo = np.array(x)#Variable independiente
bulbohumedo=np.array(y)
    # Añadir una constante para el término de intersección en el modelo
X = sm.add_constant((Tiempo))
    # Ajustar el modelo de regresión lineal utilizando OLS (Mínimos cuadrados ordinarios)
modelo = sm.OLS(bulbohumedo, X).fit()
slope1=np.around(float(modelo.params[1]),3)
plt.plot(x, modelo.predict(X), color='black',label=fr"$\beta_{1}$={slope1*10}$^\circ$C $dec^{-1}$")

#plt.plot(x, slope*x + intercept, color="black",label=fr"$\beta_{1}$={slope1*10}$^\circ$C $dec^{-1}$")
plt.plot(x,y,label="HadISD",c="red")
#plt.text(2025, 23.72, "a)", fontsize=15, color="black", ha="center", va="center", bbox=dict(facecolor="lightgray", edgecolor="black", boxstyle="round",pad=0.4))
#plt.xlim(1975,2027) 

x1=ERA_Tw_P95_mean.time
y1=ERA_Tw_P95_mean

Tiempo = np.array(x1)#Variable independiente
bulbohumedo=np.array(y1)
    # Añadir una constante para el término de intersección en el modelo
X = sm.add_constant((Tiempo))
    # Ajustar el modelo de regresión lineal utilizando OLS (Mínimos cuadrados ordinarios)
modelo = sm.OLS(bulbohumedo, X).fit()
slope1=np.around(float(modelo.params[1]),3)#.quantize(Decimal("0.01"), rounding=ROUND_DOWN)
plt.plot(x1, modelo.predict(X), color='black',label=fr"$\beta_{1}$={slope1*10}$^\circ$C $dec^{-1}$")
plt.plot(x1,y1,label="ERA5-Land",c="red",linestyle="--")
plt.grid(color='gray', alpha=0.6, linestyle='--')
plt.ylabel("$P95$ $T_{W,max}$ [$^\circ$C]",fontsize=15)
plt.legend(ncol=2,fontsize=11, loc="upper left")
plt.xticks(color="white",fontsize=12)
plt.yticks(fontsize=12)



plt.subplot(3,1,2)
x=HAD_T_P95_mean.time
y=HAD_T_P95_mean

Tiempo = np.array(x)#Variable independiente
bulbohumedo=np.array(y)
    # Añadir una constante para el término de intersección en el modelo
X = sm.add_constant((Tiempo))
    # Ajustar el modelo de regresión lineal utilizando OLS (Mínimos cuadrados ordinarios)
modelo = sm.OLS(bulbohumedo, X).fit()
slope1=np.around(float(modelo.params[1]),3)
plt.plot(x, modelo.predict(X), color='black',label=fr"$\beta_{1}$={slope1*10}$^\circ$C $dec^{-1}$")
plt.plot(x,y,label="HadISD",c="orange")

x1=ERA_T_P95_mean.time
y1=ERA_T_P95_mean

Tiempo = np.array(x1)#Variable independiente
bulbohumedo=np.array(y1)
    # Añadir una constante para el término de intersección en el modelo
X = sm.add_constant((Tiempo))
    # Ajustar el modelo de regresión lineal utilizando OLS (Mínimos cuadrados ordinarios)
modelo = sm.OLS(bulbohumedo, X).fit()
slope1=np.around(float(modelo.params[1]),3)
plt.plot(x1, modelo.predict(X), color='black',label=fr"$\beta_{1}$={slope1*10}$^\circ$C $dec^{-1}$")
plt.plot(x1,y1,label="ERA5-Land",c="orange",linestyle="--")
#plt.ylim(15.5,18.3)
plt.grid(color='gray', alpha=0.6, linestyle='--')
plt.ylabel("$P95$ $T_{max}$ [$^\circ$C]",fontsize=15)
plt.legend(ncol=2,loc="upper left",fontsize=10.5)
plt.xticks(color="white",fontsize=12)
plt.yticks(fontsize=12)
#plt.yticks((np.arange(16,19,1)),fontsize=13.5)

plt.subplot(3,1,3)
x=HAD_SH_P95_mean.time
y=HAD_SH_P95_mean*1000

Tiempo = np.array(x)#Variable independiente
bulbohumedo=np.array(y)
    # Añadir una constante para el término de intersección en el modelo
X = sm.add_constant((Tiempo))
    # Ajustar el modelo de regresión lineal utilizando OLS (Mínimos cuadrados ordinarios)
modelo = sm.OLS(bulbohumedo, X).fit()
slope1=np.around(float(modelo.params[1]),3)
plt.plot(x, modelo.predict(X), color='black',label=fr"$\beta_{1}$={slope1*10} $g$ $kg^{-1}$ $dec^{-1}$")
plt.plot(x,y,label="HadISD",c="blue")



x1=ERA_SH_P95_mean.time
y1=ERA_SH_P95_mean*1000

Tiempo = np.array(x1)#Variable independiente
bulbohumedo=np.array(y1)
    # Añadir una constante para el término de intersección en el modelo
X = sm.add_constant((Tiempo))
    # Ajustar el modelo de regresión lineal utilizando OLS (Mínimos cuadrados ordinarios)
modelo = sm.OLS(bulbohumedo, X).fit()
slope1=np.around(float(modelo.params[1]),3)
plt.plot(x1, modelo.predict(X), color='black',label=fr"$\beta_{1}$={slope1*10} $g$ $kg^{-1}$ $dec^{-1}$")
plt.plot(x1,y1,label="ERA5-Land",c="blue",linestyle="--")
#plt.ylim(15.5,18.3)
plt.grid(color='gray', alpha=0.6, linestyle='--')
plt.ylabel("$P95$ $SH_{max}$ [$g$ $kg^{-1}$]",fontsize=15)
plt.legend(ncol=2,loc="upper left",fontsize=10.5)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.tight_layout(h_pad=-0.2)

plt.savefig("P95_Max.svg",bbox_inches="tight",dpi=300)
plt.savefig("P95_Max.png",bbox_inches="tight",dpi=300)